In [48]:
# Student: Hector Banos Ramos
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME(FIXED) #####
# Change animal_shelter and AnimalShelter to match your CRUD Python module
# file name and class name
# I imported my CRUD module from the python file calLed mylib.py
from mylib import CRUD

###########################
# Data Manipulation / Model
###########################


# FIX ME(FIXED) update with your username and password and CRUD Python module name
USER = 'aacuser'
PASS = 'password1234'
HOST = 'nv-desktop-services.apporto.com'
PORT = 30325
DB = 'AAC'
COL = 'animals'

# Connect to database via CRUD Module
crud_instance = CRUD(USER, PASS, HOST, PORT, DB, COL)

# Class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
global df # The df variable is declared as a global variable so that it can be used 
          # throughout the entire app
    
#This line of code when executed at the start of the app shows unfiltered data
df = pd.DataFrame.from_records(crud_instance.read({})) # The data is around 10,000 documents

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)

# Note: To make sure that all the components work I added debug statements throughout
# the code to verify that the data is obtained correctly.

#########################
# Dashboard Layout / View
#########################


app = JupyterDash(__name__)

# FIX ME(FIXED) Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    
    # FIX ME(FIXED) Also remember to include a unique identifier such as your name or date
    # This is my unique identifier 
    html.H1("Hector Banos Ramos SNHU CS-340 MongoDB Interactive Data Table"), 
    html.Hr(),
    
    # FIX ME(FIXED) Place the HTML image tag in the line below into the app.layout
    # code according to your design
    # I aligned the logo to the center
    html.Div(style={'text-align': 'center'}, children=[
        html.A(href='https://www.snhu.edu', children=[
            # The logo includes a URL anchor tag
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                     alt='Grazioso Salvare Logo')
    ]) ]),
    
    html.Hr(),
    # FIXME(FIXED) Add in code for the interactive filtering options. For example,
    # Radio buttons, drop down, checkboxes, etc.
    # I added a drop-down menu and four filtering fuctions to control the data table and charts
    html.Div(
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='none'
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        # This line of code imports the data
        data=df.to_dict('records'),
        
        # FIXME(FIXED): Set up the features for your interactive data table to make it
        # user-friendly for your client
        # I added features so that the user can view and use the data more easily, like:
        # Limited the number of rows displayed, pagination, and sorting
        # The columns and rows are selectable and the number of elements per page is 10.
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        selected_columns=[],
        # This line to selects the first row by default
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
        # Enable single-row selection
        row_selectable="single"
    ),
    html.Br(),
    html.Hr(),
# This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
 
    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
# FIX ME(FIXED) Add code to filter interactive data table with MongoDB queries
# Each filter option has an if statement that will execute a query to retrieve the documents
# that match the search criteria established by Grazioso Salvare

    # Each database query matches the required filter functionality
    if filter_type == 'reset':
        # The reset option (returns all widgets to their original, unfiltered state)
        df_unfiltered = pd.DataFrame.from_records(crud_instance.read({}))
        df_unfiltered.drop(columns=['_id'],inplace=True)
        df = df_unfiltered.copy()
        
    elif filter_type == 'water':
        # Water Rescue filter
        query_water = {"animal_type": "Dog",
        # Preferred Breeds
        "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever",
        "Newfoundland"]},
        # Preferred Sex
        "sex_upon_outcome": "Intact Female",
        # Training Age
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
        # The current version of the collection (04/21/2024) finds 17 results
        df_water = pd.DataFrame.from_records(crud_instance.read(query_water)) 
        df_water.drop(columns=['_id'],inplace=True)
        # debug
        #print(len(df_water.to_dict(orient='records')))
        df = df_water.copy()

    elif filter_type == 'mountain':
        # Mountain or Wilderness Rescue filter
        query_mountain = {"animal_type": "Dog",
        # Preferred Breeds
        "breed": {"$in": ["German Shepherd","Alaskan Malamute","Old English Sheepdog",
        "Siberian Husky", "Rottweiler"]},
        # Preferred Sex
        "sex_upon_outcome": "Intact Male",
        # Training Age                
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
        # The current version of the collection (04/21/2024) finds 5 results
        df_mountain = pd.DataFrame.from_records(crud_instance.read(query_mountain))
        df_mountain.drop(columns=['_id'],inplace=True)
        # debug
        #print(len(df_mountain.to_dict(orient='records')))
        df = df_mountain.copy()
        
    elif filter_type == 'disaster':
        # Disaster Rescue or Individual Tracking filter
        query_disaster = {"animal_type": "Dog",
        # Preferred Breeds
        "breed": {"$in": ["Doberman Pinscher","German Shepherd","Golden Retriever",
        "Bloodhound","Rottweiler"]},
         # Preferred Sex
        "sex_upon_outcome": "Intact Male",
        # Training Age                
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
        # The current version of the collection (04/21/2024) finds 4 results
        df_disaster = pd.DataFrame.from_records(crud_instance.read(query_disaster))
        df_disaster.drop(columns=['_id'],inplace=True)
        # Debug
        #print(len(df_disaster.to_dict(orient='records')))
        df = df_disaster.copy()
        
    else:
        # The else statement is created to handle any other cases, such as when the
        # value is 'none'
        df_unfiltered = pd.DataFrame.from_records(crud_instance.read({}))
        df_unfiltered.drop(columns=['_id'],inplace=True)
        df = df_unfiltered.copy()
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
    return (data,columns)


# Display the breeds of animal based on quantity represented in the data table  
@app.callback(
    Output('graph-id', "children"),
    [Input('filter-type', 'value'),
     Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "page_current")]
)
def update_graphs(filter_type, viewData, page_current):
    ###FIX ME(FIXED) ####
    # add code for chart of your choice (e.g. pie chart) #
    # I decided to use a histogram graph, and implement if-else statements to handle
    # the way the graph would be displayed in some particular cases.
    
    if viewData is None:
        return []
    
    # debug
    #page_number_counter = page_current
    #print(page_number_counter)
    
    dff = pd.DataFrame.from_dict(viewData)
    
    if dff.empty:
        return []
    
    # Remove NaN values from the 'breed' column
    dff = dff.dropna(subset=['breed'])
    
    # At the beginning of the program execution, the dropdown menu value will be 'none' and
    # the graph will show the 10 documents on the page the user is on.
    if filter_type == 'none':
        start_index = page_current * 10
        dff = dff.iloc[start_index:start_index+10]  
        
    # If the filter type is 'reset' and the data frame has more than 9999 rows
    # and the page is different from the first one the code will print the first 10
    # documents from the first page
    elif filter_type == 'reset' and len(dff) > 9999 and page_current == 0:
        #dff = dff.head(10)
        dff    
        
    # The graph displays data in response to updates 
    return [
            dcc.Graph(

                figure=px.histogram(dff, x='breed', color='breed',
                                    title='Available Animals by Breed')
            )
        ]
    
# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
# This line of code checks if the viewData variable is either None or empty.
# It prevents further processing of the empty data, avoiding potential errors such as
# IndexError when attempting to access elements of an empty DataFrame
    if viewData is None or index is None:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    
    row = index[0] if index else 0
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    # The user will also be able to see the name of the animal
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:10487/
